# Evolving a Lunar Lander with differentiable Genetic Programming

# Setup

In [1]:
import gymnasium as gym

from genepro.node_impl import *
from genepro.evo import Evolution
from genepro.node_impl import Constant

import torch
import torch.optim as optim

import random
import os
import copy
from collections import namedtuple, deque

import matplotlib.pyplot as plt
from matplotlib import animation

In [2]:
env = gym.make("LunarLander-v2", render_mode="rgb_array")

In [3]:
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

    def __iadd__(self, other):
      self.memory += other.memory
      return self 

    def __add__(self, other):
      self.memory = self.memory + other.memory 
      return self

In [4]:
frames = []

### Creating the leaf nodes
The gym environment returns states that represent where the lunar lander is at the current frame. These states are used to calculate the next move of the lander. For each state, we create a leaf node that can be used in the tree that represent the solution.

In [5]:
# Create environment to find how many features it returns
env = gym.make("LunarLander-v2", continuous=True)
num_features = env.observation_space.shape[0]

# Create a feature for each value the evironment returns
leaf_nodes = [Feature(i) for i in range(num_features)]
# Add random constants to the features
leaf_nodes = leaf_nodes + [Constant()] 


# Evolution

## Fitness Function

Here you get to be creative. The default setup evaluates 5 episodes of 300 frames. Think of what action to pick and what fitness function to use. The Multi-tree takes an input of $n \times d$ where $n$ is a batch of size 1.

In [6]:
def fitness_function_pt(multitree, num_episodes=5, episode_duration=300, render=False, ignore_done=False):
  memory = ReplayMemory(10000)
  rewards = []

  for _ in range(num_episodes):
    # get initial state of the environment
    observation = env.reset()
    observation = observation[0]
    
    for _ in range(episode_duration):
      if render:
        frames.append(env.render())

      input_sample = torch.from_numpy(observation.reshape((1,-1))).float()
      
      # what goes here? TODO
      action = torch.argmax(multitree.get_output_pt(input_sample))
      observation, reward, terminated, truncated, info = env.step(action.item())
      rewards.append(reward)
      output_sample = torch.from_numpy(observation.reshape((1,-1))).float()
      memory.push(input_sample, torch.tensor([[action.item()]]), output_sample, torch.tensor([reward]))
      if (terminated or truncated) and not ignore_done:
        break

  fitness = np.sum(rewards)
  
  return fitness, memory

## Baseline

In [7]:
# Atomic funtions for the baseline solution
internal_nodes_base = [Plus(),Minus(),Times(),Div(),Log(),Sin(),Cos(),Exp(),Sqrt()]
leaf_nodes_base = leaf_nodes # as defined at the beginning

In [8]:
def Looper(var_to_change, runs=5):
    # runs = 5
    max_gens = 70
    Fittest =np.zeros([runs,max_gens])
    #TODO: Set to proper baseline values
    for i in range(runs):
        evo = Evolution(
            fitness_function_pt, internal_nodes_base, leaf_nodes_base, 4,
            # pop_size=256,
            # max_gens=50,
            pop_size=16,
            max_gens=2,
            max_tree_size=31,
        )
        evo.evolve()

        with open('gen.csv', 'r') as f:
            csv_reader = csv.reader(f, delimiter='\t')
            idx=0
            for row in csv_reader:
                Fittest[i, idx] =  float(row[1])
                idx+=1    
        (dt, micro) = datetime.utcnow().strftime('%Y%m%d%H%M%S.%f').split('.')
        shutil.copyfile('gen.csv', 'gen'+dt+str(int(var_to_change))+'.csv')
        os.remove('gen.csv')
    Fit_summation = np.sum(Fittest,axis=0)/runs
    print(sum(Fit_summation))

    plt.title('Fitness')
    plt.xlabel("Generations")
    plt.ylabel("Fitness")
    plt.plot(range(max_gens),Fit_summation)

    (dt, _) = datetime.utcnow().strftime('%Y%m%d%H%M%S.%f').split('.')
    plt.savefig('gen_plot_baseline'+dt+str(int(var_to_change))+'.png', format='png', dpi=300) #!!! manually set a name)
    plt.show()
    return evo

In [9]:
base_evo = Looper(1,1)
# evo = Looper(1,5)

Last Generation - Thorough Eval


TypeError: fitness_function_pt() got multiple values for argument 'num_episodes'

## Save models

In [ ]:
import pickle

with open('baseline.pickle', 'wb') as file:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(base_evo, file, pickle.HIGHEST_PROTOCOL)

# Test

In [ ]:
def get_test_score(tree):
    rewards = []

    for i in range(10):
      # get initial state
      observation = env.reset(seed=i)
      observation = observation[0]

      for _ in range(500):    
        # build up the input sample for GP
        input_sample = torch.from_numpy(observation.reshape((1,-1))).float()
        # get output (squeezing because it is encapsulated in an array)
        output = tree.get_output_pt(input_sample)
        action = torch.argmax(output)
        observation, reward, terminated, truncated, info = env.step(action.item())
        rewards.append(reward)

        output_sample = torch.from_numpy(observation.reshape((1,-1))).float()
        if (terminated or truncated):
            break

    fitness = np.sum(rewards)
    
    return fitness

best = evo.best_of_gens[-1]

print(best.get_readable_repr())
print(get_test_score(best))

SyntaxError: invalid syntax (168324889.py, line 14)

## Make an animation
Here the best evolved individual is selected and one episode is rendered. Make sure to save your lunar landers over time to track progress and make comparisons.

In [ ]:
frames = []

# gist to save gif from https://gist.github.com/botforge/64cbb71780e6208172bbf03cd9293553
def save_frames_as_gif(frames, path='./', filename='evolved_lander.gif'):
  plt.figure(figsize=(frames[0].shape[1] / 72.0, frames[0].shape[0] / 72.0), dpi=72)
  patch = plt.imshow(frames[0])
  plt.axis('off')
  def animate(i):
      patch.set_data(frames[i])
  anim = animation.FuncAnimation(plt.gcf(), animate, frames = len(frames), interval=50)
  anim.save(path + filename, writer='imagemagick', fps=60)

frames = []
fitness_function_pt(best, num_episodes=1, episode_duration=500, render=True, ignore_done=False)
env.close()
save_frames_as_gif(frames)

## Play animation

<img src="evolved_lander.gif" width="750">

## Optimisation
The coefficients in the multi-tree aren't optimised. Here Q-learning (taken from https://pytorch.org/tutorials/intermediate/reinforcement_q_learning.html) is used to optimise the weights further. Incorporate coefficient optimisation in training your agent(s). Coefficient Optimisation can be expensive. Think about how often you want to optimise, when, which individuals etc.

In [ ]:
batch_size = 128
GAMMA = 0.99

constants = best.get_subtrees_consts()

if len(constants)>0:
  optimizer = optim.AdamW(constants, lr=1e-3, amsgrad=True)

for _ in range(500):

  if len(constants)>0 and len(evo.memory)>batch_size:
    target_tree = copy.deepcopy(best)

    transitions = evo.memory.sample(batch_size)
    batch = Transition(*zip(*transitions))
    
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                        batch.next_state)), dtype=torch.bool)

    non_final_next_states = torch.cat([s for s in batch.next_state
                                               if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    state_action_values = best.get_output_pt(state_batch).gather(1, action_batch)
    next_state_values = torch.zeros(batch_size, dtype=torch.float)
    with torch.no_grad():
      next_state_values[non_final_mask] = target_tree.get_output_pt(non_final_next_states).max(1)[0].float()

    expected_state_action_values = (next_state_values * GAMMA) + reward_batch
    
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))
   
    # Optimize the model
    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_value_(constants, 100)
    optimizer.step()

print(best.get_readable_repr())
print(get_test_score(best))

In [ ]:
frames = []
fitness_function_pt(best, num_episodes=1, episode_duration=500, render=True, ignore_done=False)
env.close()
save_frames_as_gif(frames, filename='evolved_lander_RL.gif')

<img src="evolved_lander_RL.gif" width="750">